In [ ]:
import os
os.chdir("e:\GITHUB\Crime-Investigator\data")#change directory 


In [ ]:
%pwd #working directory

In [ ]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
def load_pdf_files(data):
  loader = DirectoryLoader(
    data,                     #path
    glob="*.pdf",             #only the pdfs(all pdfs)
    loader_cls=PyPDFLoader    #to load pdf documents langchain help
  )
  documents = loader.load()
  return documents

In [ ]:
extracted_data=load_pdf_files(r"e:\GITHUB\Crime-Investigator\data")

In [ ]:
extracted_data


In [ ]:
#filter to get only wanted (ex remove meta data)
from typing import List
from langchain.schema import Document

def filter_to_minimal_doc(docs: List[Document]) -> List[Document]:
    
    """Filter document objs to return a newlist of doc objects to only 
    include 'source' in metadata and page content."""

    minimal_docs: List[Document] = []
    for doc in docs:
        src= doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [ ]:
minimal_doc=filter_to_minimal_doc(extracted_data)

In [ ]:
minimal_doc

In [ ]:
#split documents into small chunks
def text_split(minimal_doc):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=20,      #to understand context with chunks
    )
  texts_chunk=text_splitter.split_documents(minimal_doc)
  return texts_chunk

In [ ]:
text_chunk=text_split(minimal_doc)
print(f"Number of chunks {len(text_chunk)}")

In [ ]:
print(text_chunk)

In [16]:
#Embeddding - using huggingface model
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
        )
    return embeddings

embedding=download_embeddings()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21000\2399630773.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21000\2399630773.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21000\2399630773.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
embedding

In [ ]:
vector=embedding.embed_query("Hello World")

In [ ]:
vector

In [11]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [12]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")
OPENROUTER_API_KEY=os.getenv("OPENROUTER_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY
os.environ["OPENROUTER_API_KEY"]=OPENROUTER_API_KEY

In [13]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
pc

In [14]:
from pinecone import ServerlessSpec 

index_name="crime-investigater"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,                   #dimension of the embedding vector
        metric="cosine",                 #similarity measure
        spec=ServerlessSpec(cloud="aws",region="us-east-1") #serverless configuration
    )

index=pc.Index(index_name)

In [17]:
from langchain_pinecone import PineconeVectorStore

# Connect to existing Pinecone index (no need to upload documents again)
docsearch = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding
)

print(f"✅ Connected to existing Pinecone index: {index_name}")

✅ Connected to existing Pinecone index: crime-investigater


In [18]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3})    

In [ ]:
retrieved_docs=retriever.invoke("First things to do in a murder case?")

In [ ]:
retrieved_docs

In [19]:
from langchain_openai import ChatOpenAI

# Configure LLM to use OpenRouter with Gemma model
llm = ChatOpenAI(
    model="google/gemma-3-12b-it:free",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7,
    max_tokens=1024
)

# Test the model
response = llm.invoke("Say hello!")
print(response.content)

Hello! 😊 It's lovely to hear from you! How are you doing today?


In [28]:
# Install missing langchain components
%pip install --upgrade langchain langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
# Install all required langchain packages
%pip install langchain[all] --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
# Alternative approach: Create RAG chain manually
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("✅ Imports successful!")

✅ Imports successful!


In [ ]:

prompt = ChatPromptTemplate.from_template(
    """You are an expert crime investigator assistant with deep knowledge of forensic procedures.
    
Context from crime investigation documents:
{context}

Question: {input}

Provide a clear, actionable answer based on the context above. Use three sentences maximum and keep the answer concise."""
)



✅ Prompt template created (Gemma-compatible)


In [ ]:
# Create RAG chain manually
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


✅ RAG chain created successfully!


In [ ]:
# Test the RAG chain with a crime investigation question
question = "What to do first in a murder scene?"
print(f"Question: {question}\n")

response = rag_chain.invoke(question)
print(f"Answer:\n{response}")

❓ Question: What to do first in a murder scene?

🔍 Answer:
In a murder scene, the first priority is to determine the victim's status – whether they are alive or deceased. This must be done before touching or moving any objects. Following this assessment, investigators should photograph and/or video the entire crime scene.


In [38]:
# Get list of installed packages to update requirements.txt
import subprocess
import sys

result = subprocess.run([sys.executable, '-m', 'pip', 'list', '--format=freeze'], 
                       capture_output=True, text=True)

# Filter only the relevant packages for this project
relevant_packages = [
    'langchain', 'langchain-openai', 'langchain-pinecone', 'langchain-community', 
    'langchain-core', 'sentence-transformers', 'pinecone', 'pypdf', 
    'python-dotenv', 'flask'
]

installed = {}
for line in result.stdout.split('\n'):
    if '==' in line:
        pkg_name, version = line.split('==')
        pkg_lower = pkg_name.lower()
        for rel_pkg in relevant_packages:
            if pkg_lower == rel_pkg.lower() or pkg_lower.replace('-', '_') == rel_pkg.replace('-', '_'):
                installed[rel_pkg] = version
                break

print("📦 Installed packages for requirements.txt:\n")
requirements_content = []
for pkg, ver in sorted(installed.items()):
    line = f"{pkg}=={ver}"
    print(line)
    requirements_content.append(line)

# Add the -e . at the end
requirements_content.append("-e .")

# Save to requirements.txt
with open(r"e:\GITHUB\Crime-Investigator\requirements.txt", 'w') as f:
    f.write('\n'.join(requirements_content))

print("\n✅ requirements.txt updated successfully!")

📦 Installed packages for requirements.txt:

flask==3.1.2
langchain==1.2.8
langchain-community==0.4.1
langchain-core==1.2.9
langchain-openai==1.1.7
langchain-pinecone==0.2.13
pinecone==7.3.0
pypdf==6.6.2
python-dotenv==1.0.1
sentence-transformers==5.2.2

✅ requirements.txt updated successfully!


## 📋 Updated requirements.txt

The [requirements.txt](../requirements.txt) file has been updated with all packages currently installed in the notebook environment:

- **flask==3.1.2** - Web framework
- **langchain==1.2.8** - LangChain core
- **langchain-community==0.4.1** - Community integrations
- **langchain-core==1.2.9** - LangChain core utilities
- **langchain-openai==1.1.7** - OpenAI/OpenRouter integration
- **langchain-pinecone==0.2.13** - Pinecone vector store
- **pinecone==7.3.0** - Pinecone client
- **pypdf==6.6.2** - PDF processing
- **python-dotenv==1.0.1** - Environment variables
- **sentence-transformers==5.2.2** - Embedding models
- **-e .** - Editable install of Crime Investigator package